Objetivo de circilar/localizar a imagem e realizar uma marcação

In [1]:
# Importações
import cv2
import numpy as np

In [2]:
# Função para obter as saídas dos nomes das camadas
def get_outputs_names(net):
    # Obter os nomes de todas as camadas da rede
    layers_names = net.getLayerNames()
    # Obter os índices das camadas de saída, i.e., as camadas com saídas não conectadas
    out_layers = net.getUnconnectedOutLayers()
    
    # Verificar se estamos lidando com uma versão mais antiga do OpenCV
    if len(out_layers.shape) > 1: 
        out_layers = out_layers.reshape(-1)

    return [layers_names[i - 1] for i in out_layers]

# Função para desenhar caixas delimitadoras nos objetos detectados
def draw_pred(frame, class_id, conf, left, top, right, bottom, classes):
    # Desenhar um retângulo ao redor do objeto detectado
    cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 3)

    label = f'{classes[class_id]}: {conf:.2f}'

    # Obter o texto tamanho
    label_size, base_line = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)

    top = max(top, label_size[1])
    cv2.rectangle(frame, (left, top - round(1.5*label_size[1])), (left + round(1.5*label_size[0]), top + base_line), (255, 255, 255), cv2.FILLED)
    cv2.putText(frame, label, (left, top), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,0), 1)

Criando um mode de escolher o veiculo principal considenrando que no fundo da imagem pode haver mais de uma veiculo, a escolha do veiculo principal tem como base o tamanho da imagem e a posição do objeto referente ao centro da foto.

In [4]:
def calculate_box_score(box, frame_width, frame_height):
    # Desempacotar as coordenadas da caixa
    left, top, width, height = box

    # Calcular a área da caixa (tamanho)
    area = width * height

    # Calcular o centro da caixa
    center_x = left + width / 2
    center_y = top + height / 2

    # Calcular a distância do centro da caixa até o centro da imagem
    center_distance = ((center_x - frame_width / 2) ** 2 + (center_y - frame_height / 2) ** 2) ** 0.5

    # Combinação de tamanho e centralidade (pode ajustar os pesos)
    score = area / center_distance  # Maior área e menor distância aumentam a pontuação

    return score

def detect_main_vehicle(frame, boxes, class_ids, confidences, classes):
    frame_height, frame_width = frame.shape[:2]

    max_score = 0
    main_vehicle_idx = -1

    for i, box in enumerate(boxes):
        if classes[class_ids[i]] == "car" or classes[class_ids[i]] == "truck":  # Ajuste conforme necessário
            score = calculate_box_score(box, frame_width, frame_height)

            if score > max_score:
                max_score = score
                main_vehicle_idx = i

    if main_vehicle_idx != -1:
        box = boxes[main_vehicle_idx]
        left, top, width, height = box
        draw_pred(frame, class_ids[main_vehicle_idx], confidences[main_vehicle_idx], left, top, left + width, top + height, classes)
    
    return frame

In [5]:
boxes_global = [] # declarando a variavel global para pegar a list do boxes

def detect_objects(image_path):
    
    global boxes_global
    # Carregar os nomes das classes
    classes_file = "D:\Pos_IA\TCC\yolov3-8\data\coco.names"
    with open(classes_file, 'rt') as f:
        classes = f.read().rstrip('\n').split('\n')

    # Carregar a imagem
    frame = cv2.imread(image_path)
    
    # Verificar se a imagem foi carregada corretamente
    if frame is None:
        print(f"Erro ao carregar a imagem: {image_path}")
        return None


    # Caminho para os arquivos de configuração e pesos do YOLO
    model_configuration = "D:\Pos_IA\TCC\yolov3-8\cfg\yolov3.cfg"
    model_weights = "D:\Pos_IA\TCC\yolov3-8\weights\yolov3.weights"

    # Inicializar a rede neural
    net = cv2.dnn.readNetFromDarknet(model_configuration, model_weights)
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

    # Criar um blob da imagem e passar pelo modelo
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(get_outputs_names(net))

    # Processamento das saídas
    frame_height = frame.shape[0]
    frame_width = frame.shape[1]
    class_ids = []
    confidences = []
    boxes = []
    conf_threshold = 0.5
    nms_threshold = 0.4

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf_threshold:
                center_x = int(detection[0] * frame_width)
                center_y = int(detection[1] * frame_height)
                width = int(detection[2] * frame_width)
                height = int(detection[3] * frame_height)
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])

    #----- com essa parte do codigo eu consigo localizar todos os veiculos da imagem
    # # Aplicar non-maxima suppression para remover caixas delimitadoras redundantes
    # indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

    # # Verificar a forma do array 'indices' e ajustar o loop de acordo
    # if len(indices) > 0 and len(indices.shape) == 1:
    #     # Se 'indices' for um array 1D (caso comum em versões mais recentes do OpenCV)
    #     for i in indices:
    #         box = boxes[i]
    #         left = box[0]
    #         top = box[1]
    #         width = box[2]
    #         height = box[3]
    #         draw_pred(frame, class_ids[i], confidences[i], left, top, left + width, top + height, classes)
    # else:
    #     # Se 'indices' for um array 2D (como em algumas versões mais antigas do OpenCV)
    #     for i in indices:
    #         i = i[0]
    #         box = boxes[i]
    #         left = box[0]
    #         top = box[1]
    #         width = box[2]
    #         height = box[3]
    #         draw_pred(frame, class_ids[i], confidences[i], left, top, left + width, top + height, classes)
    
    # Chamar detect_main_vehicle para destacar o veículo principal
    frame = detect_main_vehicle(frame, boxes, class_ids, confidences, classes)
    boxes_global = boxes
    
    return frame


In [7]:
# Supondo que você tenha uma imagem de teste, você pode chamá-la assim:
result_image = detect_objects('D:\Pos_IA\TCC\carrob.jpg')
# Salvar a imagem combinada
cv2.imwrite('Image_com_deteccao.png', result_image)

# Verificar se a imagem resultante é válida antes de exibi-la
if result_image is not None:
    cv2.imshow("Detected Objects", result_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Não foi possível processar a imagem.")


Função para que seja apagado o fundo e colocar uma outra cor para que seja melhor na identificação do objeto na comparação

In [8]:
def isolate_vehicle(image, box, fill_color=(255, 255, 5)):
    """
    Isola o veículo na imagem, alterando o fundo para uma cor única.

    Args:
    - image: Imagem original.
    - box: Caixa delimitadora do veículo principal (x, y, width, height).
    - fill_color: Cor do fundo (padrão Laranja florecente).

    Returns:
    - Imagem com o veículo isolado.
    """
    # Criar uma cópia da imagem
    isolated_image = image.copy()

    # Criar uma máscara preta do mesmo tamanho da imagem
    mask = np.zeros_like(image)

    # Obter as coordenadas da caixa delimitadora
    x, y, width, height = box

    # Preencher a área do veículo na máscara com branco
    mask[y:y+height, x:x+width] = [255, 255, 255]

    # Aplicar a máscara para isolar o veículo
    # isolated_image[np.where(mask == 0)] = fill_color

    for c in range(3):  # Para cada canal de cor (RGB)
        isolated_image[:, :, c][mask[:, :, c] == 0] = fill_color[c]

    return isolated_image


In [20]:
boxes_global

[[1001, 14, 270, 204],
 [122, 32, 1078, 648],
 [165, 34, 999, 772],
 [205, 37, 981, 762],
 [146, 48, 1038, 811],
 [166, 58, 1059, 792],
 [1015, -10, 263, 229],
 [1020, 0, 252, 232]]

In [9]:
# box = boxes_global
#  Recorta e pinta o fundo da imagem de outra cor para facilicar a comparação dos objetos 
if boxes_global:
    # Seleciona a caixa com a maior área
    box = max(boxes_global, key=lambda b: b[2] * b[3])
else:
    print("Nenhuma caixa foi detectada.")
    box = None
if box:
    # Isolar o veículo na imagem
    isolated_vehicle_image = isolate_vehicle(result_image, box)

    # Exibir a imagem resultante
    cv2.imshow("Isolated Vehicle", isolated_vehicle_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Opcionalmente, salvar a imagem resultante
    output_path = 'caminho_para_salvar_imagem_isolada.jpg'
    cv2.imwrite(output_path, isolated_vehicle_image)
else:
    print("Não foi possível isolar o veículo. A caixa delimitadora está vazia.")

# ficou bom os traços 

In [10]:
# ficou bom os traços 

# cria um contor e identifica traços dentro do contorno criado 

def draw_precise_contour(image, box):
    # Extrair a região de interesse (ROI) com base na caixa delimitadora
    x, y, w, h = box
    roi = image[y:y+h, x:x+w]

    # Converter a ROI para escala de cinza e aplicar detecção de bordas
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)

    # Encontrar contornos na ROI
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Desenhar os contornos na imagem original
    cv2.drawContours(image, contours, -1, (0, 255, 0), 3, offset=(x,y))

    return image

# Supondo que 'image' seja sua imagem original e 'box' a caixa delimitadora
image_with_contour = draw_precise_contour(result_image, box)

# Exibir a imagem resultante
cv2.imshow("Image with Precise Contour", image_with_contour)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
# Supondo que 'image' seja a imagem original e 'box' a caixa delimitadora
# image = 'D:\Pos_IA\TCC\carrob.jpg'
# image = cv2.imread(image)
image = result_image

x, y, w, h = box  # Desempacotar a caixa delimitadora
roi = image[y:y+h, x:x+w]  # Extrair a região de interesse

# Converter para escala de cinza
gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

# Aplicar um desfoque para reduzir o ruído
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Detecção de bordas Canny
edges = cv2.Canny(blurred, 50, 150)

# Encontrar contornos
contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filtrar contornos, se necessário, por exemplo, pelo tamanho
contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 100]

# Desenhar contornos na imagem original
cv2.drawContours(image, contours, -1, (0, 255, 0), 2, offset=(x, y))

# Exibir a imagem com contornos
cv2.imshow("Vehicle Contours", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
image = 'D:\Pos_IA\TCC\carrob.jpg'
image = cv2.imread(image)

# Supondo que 'image' seja a imagem original e 'box' a caixa delimitadora
x, y, w, h = box  # Desempacotar a caixa delimitadora do YOLO
roi = image[y:y+h, x:x+w]  # Extrair a região de interesse

# Processamento da ROI para encontrar contornos
gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edges = cv2.Canny(blurred, 50, 150)

# Encontrar contornos na ROI processada
contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 10]  # Filtrar contornos por área

# Desenhar contornos filtrados na imagem original
for cnt in contours:
    # Ajustar as coordenadas do contorno para a posição original na imagem
    cnt += np.array([x, y])
    cv2.drawContours(image, [cnt], -1, (0, 255, 0), 2)

# Exibir a imagem com contornos
cv2.imshow("Vehicle Contours", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Salvar a imagem resultante
cv2.imwrite('imagem_com_contornos.jpg', image)


True